In [2]:
import random
import torch
from d2l import torch as d2l

In [3]:
# 生成数据集
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [4]:
# 读取数据集
# 定义一个函数，打乱数据集中的样本并以小批量方式获取数据
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[ 1.5874, -0.8020],
        [ 0.5569, -0.4273],
        [ 0.3662, -0.1050],
        [ 0.8983, -2.5842],
        [ 2.4951, -0.4819],
        [ 1.5066,  0.6311],
        [ 0.3291,  0.4193],
        [ 1.3072, -1.3554],
        [ 0.2108,  0.5160],
        [ 0.1687,  0.6073]]) 
 tensor([[10.0870],
        [ 6.7720],
        [ 5.2672],
        [14.7819],
        [10.8221],
        [ 5.0714],
        [ 3.4122],
        [11.4165],
        [ 2.8543],
        [ 2.4722]])


In [5]:
# 初始化模型参数
# 通过从均值为0、标准差为0.01的正态分布中采样随机数来初始化权重，并将偏置初始化为0
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# 初始化参数之后，接下来的任务是更新这些参数，直到这些参数足够拟合数据
# 每次更新都需要计算损失函数关于模型参数的梯度

In [6]:
# 定义模型
# 定义模型，将模型的输入和参数同模型的输出关联起来
def linreg(X, w, b):
    '''线性回归模型'''
    return torch.matmul(X, w) + b

In [7]:
# 定义损失函数
def squared_loss(y_hat, y):
    '''均方损失'''
    return (y_hat - y.reshape(y_hat.shape))**2/2

In [8]:
# 定义优化算法
def sgd(params, lr, batch_size):
    '''小批量随机梯度下降'''
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad/batch_size
            param.grad.zero_()

In [9]:
# 训练
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch{epoch + 1}, loss{float(train_l.mean()):f}')

epoch1, loss0.039722
epoch2, loss0.000145
epoch3, loss0.000049
